In [116]:
%load_ext autoreload
%autoreload 2

import stravaio
import sweat
import dotenv
import pandas as pd
import plotly_express as px
dotenv.load_dotenv()
import plotly.graph_objs as go
import plotly.plotly as py
from plotly.offline import iplot, init_notebook_mode

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
from sweat.metrics import rolling_mean

In [6]:
token = stravaio.strava_oauth2()

2019-07-02 14:16:59.514 | INFO     | stravaio:strava_oauth2:343 - serving at port 8000
2019-07-02 14:17:10.506 | DEBUG    | stravaio:run_server_and_wait_for_token:397 - code: 1261568f34646ade4f25d91b29cfd44fc4b42cd4
2019-07-02 14:17:12.293 | DEBUG    | stravaio:run_server_and_wait_for_token:406 - Authorized athlete: 12cdf4bf442fd1634b4ee2a470007d836a91ed00


In [9]:
access_token = token['access_token']

In [10]:
client = stravaio.StravaIO(access_token)

In [17]:
athlete = client.get_logged_in_athlete()

In [12]:
list_activities = client.get_logged_in_athlete_activities(after="2019-06-25")

Fetched 6, the latests is on 2019-07-02 09:27:51+00:00


In [13]:
[print(f"{a.id}: {a.name}") for a in list_activities]

2478368148: Morning Ride
2484463522: One Hour Lower Back Routine
2488860384: Morning Ride
2491572855: Morning Ride
2494619456: One Hour Lower Back Routine
2497323866: SUF Strength - Intermediate Level 3A


[None, None, None, None, None, None]

In [14]:
activity = client.get_activity_by_id(2491572855)

In [15]:
activity

Activity: 2491572855, Date: 2019-06-30 03:24:11+00:00, Name: Morning Ride

In [18]:
stream = client.get_activity_streams(athlete_id=athlete.id, id=2491572855)

In [285]:
df = pd.DataFrame(stream.to_dict())
df.head()

,time,distance,altitude,velocity_smooth,heartrate,cadence,watts,temp,moving,grade_smooth,lat,lng
0,289,1378.6,444.8,6.2,111,82,150.0,18,True,2.6,47.086439,15.424913
1,290,1384.2,445.0,6.1,111,83,121.0,18,True,2.7,47.086484,15.424880
2,291,1389.6,445.2,5.9,112,81,97.0,18,True,2.9,47.086523,15.424838
3,292,1394.9,445.4,5.7,112,83,83.0,18,True,3.0,47.086555,15.424786
4,293,1399.6,445.4,5.4,113,83,0.0,18,True,2.0,47.086580,15.424735


# Preprocess

In [302]:
window=30
df['smooth_watts'] = sweat.metrics.rolling_mean(df.watts, window)
df['smooth_heartrate'] = sweat.metrics.rolling_mean(df.heartrate, window)
df['smooth_cadence'] = sweat.metrics.rolling_mean(df.cadence, window)

In [303]:
e4DP = [-1, 200, 257, 326, 444, 983]
e4DP_names = ['END', 'FTP', 'MAP', 'AC', 'NM']
e4DP_colors = [None,
               px.colors.sequential.dense[1],
               px.colors.sequential.dense[3],
               px.colors.sequential.dense[6],
               px.colors.sequential.dense[8],
               px.colors.sequential.dense[11]]

In [304]:
df['power_zones'] = sweat.metrics.compute_zones(df.smooth_watts, zones=e4DP)

In [305]:
time_in_power_zones = sweat.metrics.time_in_zones(df.smooth_watts, zones=e4DP)
time_in_power_zones

array([8591, 2077,  434,    3,    0], dtype=int64)

In [306]:
df['hr_zones'] = sweat.metrics.compute_zones(df.smooth_heartrate, lthr=165)

In [307]:
time_in_hr_zones = sweat.metrics.time_in_zones(df.smooth_heartrate, lthr=165)
time_in_hr_zones

array([3282, 4659, 2230,  934,    0], dtype=int64)

# Plot

In [308]:
def watts_to_plot(df, power_zone=1):
    watts = df.smooth_watts.copy()
    watts[df.power_zones==power_zone] = df.smooth_watts[df.power_zones==power_zone]
    watts[df.power_zones!=power_zone] = 0
    return watts

In [309]:
def make_trace(df, power_zone):
    rv = go.Scatter({
            'x': df.time,
            'y': watts_to_plot(df, power_zone),
            'fill':'tozeroy',
            'mode':'none',
            'fillcolor': e4DP_colors[power_zone],
            'name': e4DP_names[power_zone-1]
        })
    return rv

data = [make_trace(df, power_zone) for power_zone in range(1, len(e4DP_names))]

fig = go.Figure(data=data)
iplot(fig)